In [48]:
# rag_with_my_model.py
import torch
import fitz  # PyMuPDF
import re
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import sentencepiece as spm
from decoder_only_gpt import My_GPT_model

## HIGH LEVEL RAG FLOW (1 line)
#### Text → Tokens → Embeddings → Retrieval → Tokens → LLM → Tokens → Text

In [49]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Load tokenizer & model (tera trained model)
sp = spm.SentencePieceProcessor()
sp.load("hindi_tokenizer_new.model")

True

In [50]:
model = My_GPT_model(
    vocab_size=sp.get_piece_size(),
    num_layers=12,
    d_model=512,
    d_ff=2048,
    num_heads=8,
    seq_len=512
).to(DEVICE)

# Load final SFT checkpoint
model.load_state_dict(torch.load("full_sft_epoch4_step10000.pt", map_location=DEVICE))
model.eval()

My_GPT_model(
  (decoder): Decoder(
    (embedding): Embedding(32768, 512)
    (layers): ModuleList(
      (0-11): 12 x Decoder_GPT_Block(
        (swi_glu): SwiGLU_FFN(
          (w1): Linear(in_features=512, out_features=1536, bias=False)
          (w2): Linear(in_features=512, out_features=1536, bias=False)
          (w3): Linear(in_features=1536, out_features=512, bias=False)
          (act): SiLU()
        )
        (masked_mha): Masked_MHA(
          (Q): Linear(in_features=512, out_features=512, bias=True)
          (K): Linear(in_features=512, out_features=512, bias=True)
          (V): Linear(in_features=512, out_features=512, bias=True)
          (fc_out): Linear(in_features=512, out_features=512, bias=True)
        )
        (rms_norm0): RMSNorm()
        (rms_norm1): RMSNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=512, out_features=32768, bias=False)
)

In [4]:
# 2. Load embedding & reranker
embed_model = SentenceTransformer("intfloat/multilingual-e5-base", device=DEVICE)
reranker = CrossEncoder("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1", device=DEVICE)

In [51]:
print("Vocab size from tokenizer:", sp.get_piece_size())
print("Model vocab size:", model.lm_head.out_features)  # should match

Vocab size from tokenizer: 32768
Model vocab size: 32768


#### STEP 1 — USER QUERY ENTERS SYSTEM

In [52]:
query = "महात्मा गांधी का जन्म कब हुआ था?\n�"

#### STEP 2 — clean_hindi_text

In [53]:
def clean_hindi_text(text):
    if not text:
        return ""

    # Remove non-printable characters
    text = re.sub(r'[\x00-\x1F\x7F]', ' ', text)

    # Fix common PDF junk chars
    text = re.sub(r'[�•ﬁﬂ–—]', ' ', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [54]:
text = re.sub(r'[\x00-\x1F\x7F]', ' ', query)
text

'महात्मा गांधी का जन्म कब हुआ था? �'

In [55]:
text = re.sub(r'[�•ﬁﬂ–—]', ' ', text)
text

'महात्मा गांधी का जन्म कब हुआ था?  '

In [56]:
text = re.sub(r'\s+', ' ', text).strip()
text

'महात्मा गांधी का जन्म कब हुआ था?'

In [57]:
cleaned_query = clean_hindi_text(query)
cleaned_query

'महात्मा गांधी का जन्म कब हुआ था?'

#### STEP 3 — cleaned QUERY → EMBEDDING

In [58]:
query_embedding = embed_model.encode(cleaned_query)

In [59]:
query_embedding

array([ 2.64983140e-02,  6.60813600e-02, -9.58607253e-03,  2.82110553e-02,
        2.61246618e-02, -4.40113023e-02, -2.21675746e-02, -4.27232273e-02,
        1.46573177e-02,  1.20486198e-02, -3.63032031e-03,  3.98490466e-02,
        1.83519334e-01,  4.25617173e-02, -2.68076733e-02, -2.99928412e-02,
        5.37063368e-02, -2.49282252e-02,  3.86897922e-02,  2.32586823e-02,
        6.06049448e-02, -5.78955188e-03,  8.08594674e-02, -3.15969973e-03,
       -2.60141063e-02,  7.31334370e-03,  1.98799949e-02,  1.12127978e-02,
        3.69927883e-02, -5.56596601e-03,  4.71094213e-02, -4.71728779e-02,
        4.56942506e-02, -2.16569938e-02,  2.51096636e-02,  2.19076127e-02,
       -1.27007673e-03, -1.02402447e-02,  1.95584614e-02,  5.13352491e-02,
        1.45036762e-03,  1.38823027e-02,  1.31985929e-03, -6.22888468e-02,
        2.69659422e-02,  2.55390978e-03,  2.18719654e-02,  1.10848378e-02,
       -1.26092904e-03, -5.49523123e-02,  1.27634201e-02, -1.14831831e-02,
        1.13381566e-02,  

In [60]:
print("Embedding shape:", query_embedding.shape)  # (768,)
print("Embedding dtype:", query_embedding.dtype)  # float32
print("Embedding snippet:", query_embedding[:5])

Embedding shape: (768,)
Embedding dtype: float32
Embedding snippet: [ 0.02649831  0.06608136 -0.00958607  0.02821106  0.02612466]


In [61]:
def fix_pdf_text(raw_text):
    text = re.sub(r'\s+', ' ', raw_text)
    text = re.sub(r'([ऀ-ॿ])([A-Za-z0-9])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z0-9])([ऀ-ॿ])', r'\1 \2', text)
    return text.strip()

In [62]:
pdf_path = "doc.pdf"
doc = fitz.open(pdf_path)

raw_text = ""
for page in doc:
    raw_text += page.get_text("text")

doc.close()

fixed_text = fix_pdf_text(raw_text)
cleaned_text = clean_hindi_text(fixed_text)

print(f"Fixed + Cleaned text snippet:\n{cleaned_text[:500]}")

Fixed + Cleaned text snippet:
महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीक


In [63]:
cleaned_text

'महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीका गांधीजी ने भारत में मैट्रिक तक पढ़ाई की और फिर लंदन जाकर बैरिस्टर की डिग्री प्राप्त की। 1893 में वे दक्षिण अफ्रीका गए, जहां उन्होंने भारतीयों पर हो रहे नस्लीय भेदभाव का सामना किया। एक ट्रेन में उन्हें प्रथम श्रेणी के डिब्बे से बाहर फेंक दिया गया, जिसने उनके जीवन की दिशा बदल दी। दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया। उन्होंने भारतीयों के अधिकारों के लिए अहिंसक आंदोलन चलाए, जैसे नेटाल इंडियन कांग्रेस की स्थापना और ट्रांसवाल में विरोध। यहां उन्होंने अहिंसा, सत्य और सविन

In [64]:
# from langchain.document_loaders import PyMuPDFLoader

# loader = PyMuPDFLoader("doc.pdf")
# docs = loader.load()

# docs

# def fix_hindi_spacing(text):
#     text = re.sub(r'([\u0900-\u097F])([\u0900-\u097F])', r'\1 \2', text)
#     text = re.sub(r'\s+', ' ', text)
#     return text

# for d in docs:
#     print(docs[0].page_content[:100])

# clean_hindi_pdf_text(d.page_content)

#### STEP 4 — VECTOR SEARCH (RETRIEVAL)

In [65]:
#Chunking function (sentence level)
def chunk_text(text, max_tokens=250, overlap=50):
    tokens = sp.encode(text)
    chunks = []
    i = 0
    while i < len(tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(sp.decode(chunk))
        i += max_tokens - overlap
    return chunks

In [66]:
def sentence_chunk_text(text, max_chars=800, overlap_sentences=1):
    sentences = re.split(r'(?<=[।!?])\s+', text)

    chunks = []
    current_sents = []

    for sent in sentences:
        current_sents.append(sent)

        chunk_text = " ".join(current_sents)

        if len(chunk_text) >= max_chars:
            chunks.append(chunk_text.strip())

            # overlap: last N full sentences
            current_sents = current_sents[-overlap_sentences:]

    if current_sents:
        chunks.append(" ".join(current_sents).strip())

    return chunks

In [67]:
#Build FAISS index
def build_index(chunks):
    texts = ["passage: " + chunk for chunk in chunks]
    embeddings = embed_model.encode(texts, normalize_embeddings=True, batch_size=32)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings.astype(np.float32))
    return index, chunks

In [68]:
chunks = sentence_chunk_text(cleaned_text, max_chars=800, overlap_sentences=1)

In [69]:
chunks

['महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीका गांधीजी ने भारत में मैट्रिक तक पढ़ाई की और फिर लंदन जाकर बैरिस्टर की डिग्री प्राप्त की। 1893 में वे दक्षिण अफ्रीका गए, जहां उन्होंने भारतीयों पर हो रहे नस्लीय भेदभाव का सामना किया। एक ट्रेन में उन्हें प्रथम श्रेणी के डिब्बे से बाहर फेंक दिया गया, जिसने उनके जीवन की दिशा बदल दी। दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया।',
 'दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया। उन्होंने भारतीयों के अधिकारों के लिए अहिंसक आंदोलन चलाए, जैसे नेटाल इंडियन कांग्रेस

In [70]:
texts = ["passage: " + chunk for chunk in chunks]
texts

['passage: महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीका गांधीजी ने भारत में मैट्रिक तक पढ़ाई की और फिर लंदन जाकर बैरिस्टर की डिग्री प्राप्त की। 1893 में वे दक्षिण अफ्रीका गए, जहां उन्होंने भारतीयों पर हो रहे नस्लीय भेदभाव का सामना किया। एक ट्रेन में उन्हें प्रथम श्रेणी के डिब्बे से बाहर फेंक दिया गया, जिसने उनके जीवन की दिशा बदल दी। दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया।',
 'passage: दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया। उन्होंने भारतीयों के अधिकारों के लिए अहिंसक आंदोलन चलाए, जैसे नेट

In [71]:
embeddings = embed_model.encode(texts, normalize_embeddings=True, batch_size=32)
embeddings

array([[ 4.3732967e-02,  7.8437299e-02, -6.8676681e-03, ...,
        -7.4431017e-02, -4.4231549e-02,  2.6142413e-02],
       [ 6.4616841e-03,  6.1080024e-02, -8.8326633e-03, ...,
        -4.1475568e-02, -3.7295274e-02,  3.6302570e-02],
       [ 7.9807432e-05,  6.9044627e-02, -5.4570683e-03, ...,
        -4.9108285e-02, -5.6223407e-02,  2.9995838e-02],
       [-8.5158283e-03,  5.7823349e-02, -5.9552589e-03, ...,
        -6.6032328e-02, -4.8739627e-02,  4.3646973e-02]],
      shape=(4, 768), dtype=float32)

In [72]:
dim = embeddings.shape[1]
dim

768

In [73]:
index = faiss.IndexFlatIP(dim)
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x000002DD62695920> >

In [74]:
index.add(embeddings.astype(np.float32))

In [75]:
index, chunks = build_index(chunks)

In [76]:
query_embedding = embed_model.encode([cleaned_query])

In [77]:
query_embedding

array([[ 2.64983140e-02,  6.60813600e-02, -9.58607253e-03,
         2.82110553e-02,  2.61246618e-02, -4.40113023e-02,
        -2.21675746e-02, -4.27232273e-02,  1.46573177e-02,
         1.20486198e-02, -3.63032031e-03,  3.98490466e-02,
         1.83519334e-01,  4.25617173e-02, -2.68076733e-02,
        -2.99928412e-02,  5.37063368e-02, -2.49282252e-02,
         3.86897922e-02,  2.32586823e-02,  6.06049448e-02,
        -5.78955188e-03,  8.08594674e-02, -3.15969973e-03,
        -2.60141063e-02,  7.31334370e-03,  1.98799949e-02,
         1.12127978e-02,  3.69927883e-02, -5.56596601e-03,
         4.71094213e-02, -4.71728779e-02,  4.56942506e-02,
        -2.16569938e-02,  2.51096636e-02,  2.19076127e-02,
        -1.27007673e-03, -1.02402447e-02,  1.95584614e-02,
         5.13352491e-02,  1.45036762e-03,  1.38823027e-02,
         1.31985929e-03, -6.22888468e-02,  2.69659422e-02,
         2.55390978e-03,  2.18719654e-02,  1.10848378e-02,
        -1.26092904e-03, -5.49523123e-02,  1.27634201e-0

In [87]:
D, I = index.search(query_embedding, k=1)

In [88]:
D

array([[0.87355447]], dtype=float32)

In [89]:
I

array([[0]])

In [90]:
retrieved_chunks = [chunks[i] for i in I[0]]

for idx, ch in zip(I[0], retrieved_chunks):
    print(f"\n--- Chunk {idx} ---\n{ch}\n")


--- Chunk 0 ---
महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीका गांधीजी ने भारत में मैट्रिक तक पढ़ाई की और फिर लंदन जाकर बैरिस्टर की डिग्री प्राप्त की। 1893 में वे दक्षिण अफ्रीका गए, जहां उन्होंने भारतीयों पर हो रहे नस्लीय भेदभाव का सामना किया। एक ट्रेन में उन्हें प्रथम श्रेणी के डिब्बे से बाहर फेंक दिया गया, जिसने उनके जीवन की दिशा बदल दी। दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया।



In [91]:
context_ids = []
for chunk in retrieved_chunks:
    context_ids += sp.encode("[संदर्भ] " + chunk + "\n")

In [92]:
print("Context tokens:", len(context_ids))

Context tokens: 195


In [93]:
context_ids

[28843,
 1,
 28851,
 123,
 30,
 28875,
 1,
 5209,
 1187,
 26,
 16462,
 5209,
 1187,
 28879,
 2774,
 10457,
 158,
 19058,
 149,
 376,
 15,
 28879,
 329,
 11,
 3767,
 10319,
 11,
 499,
 1237,
 950,
 241,
 28869,
 834,
 986,
 401,
 4127,
 3849,
 16809,
 5056,
 1187,
 132,
 28869,
 1424,
 44,
 7192,
 797,
 5209,
 1187,
 40,
 1424,
 2704,
 2227,
 12589,
 28948,
 28940,
 28,
 2174,
 11,
 17371,
 28864,
 996,
 22,
 382,
 132,
 28869,
 398,
 1285,
 16809,
 5056,
 1187,
 17371,
 28864,
 996,
 546,
 11,
 12317,
 241,
 44,
 1959,
 8596,
 119,
 4795,
 3004,
 8194,
 26,
 525,
 1382,
 28869,
 14232,
 4380,
 32,
 107,
 2789,
 28879,
 12301,
 44,
 12293,
 11,
 339,
 17591,
 2439,
 241,
 28869,
 17322,
 532,
 26,
 2263,
 22,
 834,
 2654,
 22270,
 32,
 55,
 134,
 28869,
 1082,
 44,
 2089,
 4353,
 14232,
 51,
 329,
 22,
 14840,
 181,
 3021,
 26,
 44,
 445,
 4804,
 2088,
 13065,
 11842,
 26,
 2747,
 973,
 26,
 28869,
 12589,
 28940,
 28937,
 22,
 306,
 2089,
 4353,
 295,
 28879,
 873,
 271,
 6708,
 46,
 5

In [94]:
sp.decode(context_ids)

' ⁇ संदर्भ ⁇  महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीका गांधीजी ने भारत में मैट्रिक तक पढ़ाई की और फिर लंदन जाकर बैरिस्टर की डिग्री प्राप्त की। 1893 में वे दक्षिण अफ्रीका गए, जहां उन्होंने भारतीयों पर हो रहे नस्लीय भेदभाव का सामना किया। एक ट्रेन में उन्हें प्रथम श्रेणी के डिब्बे से बाहर फेंक दिया गया, जिसने उनके जीवन की दिशा बदल दी। दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया। ⁇ '

In [111]:
prompt_ids = (
    sp.encode("""
सिर्फ़ नीचे दिए संदर्भ से ही जवाब दो।
अगर संदर्भ में उत्तर नहीं है तो लिखो: "संदर्भ में जानकारी नहीं है।"
""")
    + context_ids
    + sp.encode(f"\nप्रश्न: {query}\nउत्तर:")
)

In [112]:
print("Prompt text:", sp.decode(prompt_ids))
print("Prompt length:", len(prompt_ids))

Prompt text:  ⁇ सिर्फ़ नीचे दिए संदर्भ से ही जवाब दो। ⁇ अगर संदर्भ में उत्तर नहीं है तो लिखो: "संदर्भ में जानकारी नहीं है।" ⁇   ⁇ संदर्भ ⁇  महात्मा गांधी की जीवनी महात्मा गांधी, जिन्हें बापू या राष्ट्रपिता कहा जाता है, भारत के स्वतंत्रता संग्राम के सबसे प्रमुख नेता थे। उनका पूरा नाम मोहनदास करमचंद गांधी था। जन्म और प्रारंभिक जीवन महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के पोरबंदर में हुआ था। उनके पिता करमचंद गांधी पोरबंदर राज्य के दीवान थे और माता पुतलीबाई धार्मिक प्रवृत्ति की महिला थीं। गांधीजी बचपन से ही सत्य, अहिंसा और नैतिकता के प्रति झुकाव रखते थे। 13 वर्ष की आयु में उनका विवाह कस्तूरबा से हो गया। शिक्षा और दक्षिण अफ्रीका गांधीजी ने भारत में मैट्रिक तक पढ़ाई की और फिर लंदन जाकर बैरिस्टर की डिग्री प्राप्त की। 1893 में वे दक्षिण अफ्रीका गए, जहां उन्होंने भारतीयों पर हो रहे नस्लीय भेदभाव का सामना किया। एक ट्रेन में उन्हें प्रथम श्रेणी के डिब्बे से बाहर फेंक दिया गया, जिसने उनके जीवन की दिशा बदल दी। दक्षिण अफ्रीका में गांधीजी ने पहली बार सत्याग्रह का प्रयोग किया। ⁇   ⁇ प्रश्न: 

In [97]:
@torch.no_grad()
def generate_answer_from_ids(
    prompt_ids,
    max_new_tokens: int = 300,
    temperature: float = 0.6,
    top_p: float = 0.85,
    repetition_penalty: float = 1.4,
    penalty_window: int = 64,
    max_total_length: int = 512,
    EOS_ID: int = 2  # Example EOS token id, अपने tokenizer के हिसाब से बदलें
) -> str:
    """
    Generate answer from prompt token ids using nucleus sampling.
    Returns only the newly generated text after the prompt.
    """

    if not prompt_ids:
        print("Empty prompt!")
        return ""

    input_ids = torch.tensor([prompt_ids], dtype=torch.long, device=DEVICE)
    generated = input_ids.clone()

    for step in range(max_new_tokens):
        current_len = generated.shape[1]

        # Sliding window to avoid sequence length overflow
        if current_len >= max_total_length:
            generated = generated[:, -max_total_length:]
            print(f"Truncated generated tokens to last {max_total_length}")

        # Forward pass to get logits: shape (1, seq_len, vocab_size)
        logits = model(generated)

        # Pick logits of the last token only
        next_logits = logits[:, -1, :].clone()

        # Repetition penalty: penalize tokens recently generated
        if repetition_penalty > 1.0:
            recent_tokens = generated[0, -min(penalty_window, current_len):].tolist()
            for token_id in set(recent_tokens):
                if token_id < next_logits.size(-1):
                    # Penalize by dividing logits for repeated tokens
                    next_logits[0, token_id] /= repetition_penalty

        # Temperature scaling
        if temperature != 1.0:
            next_logits = next_logits / temperature

        # Convert logits to probabilities
        probs = torch.softmax(next_logits, dim=-1)

        # Top-p (nucleus) filtering
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

        # Remove tokens with cumulative prob above top_p threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift mask right by 1 to keep first token above threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = False

        # Set probabilities of removed tokens to zero
        sorted_probs[sorted_indices_to_remove] = 0.0

        # Normalize probabilities again
        prob_sum = sorted_probs.sum()
        if prob_sum.item() == 0:
            # Fallback to greedy if all filtered out (rare case)
            next_token = torch.argmax(probs, dim=-1, keepdim=True)
            print("All probabilities zero after top-p filtering; fallback to greedy.")
        else:
            sorted_probs /= prob_sum
            next_token_idx = torch.multinomial(sorted_probs, num_samples=1)
            next_token = sorted_indices.gather(-1, next_token_idx)

        # Append the new token to generated sequence
        generated = torch.cat([generated, next_token], dim=1)

        # Decode the token for debug printing
        next_token_id = next_token.item()
        token_text = sp.decode([next_token_id])
        # print(f"Step {step + 1}: Generated token id = {next_token_id}, Text = '{token_text}'")

        # Stop if EOS token generated
        if next_token_id == EOS_ID:
            print("EOS token generated, stopping generation.")
            break

    # Extract tokens generated after prompt
    answer_ids = generated[0, len(prompt_ids):].tolist()

    try:
        decoded_answer = sp.decode(answer_ids).strip()
    except Exception as e:
        print(f"Decoding error: {e}")
        decoded_answer = ""

    return decoded_answer


In [98]:
test_text = "नमस्ते"
test_ids = sp.encode(test_text)
print("Encoded tokens:", test_ids)
print("Decoded back:", sp.decode(test_ids))

Encoded tokens: [20776]
Decoded back: नमस्ते


In [99]:
print("Prompt text:", test_text)
print("Prompt tokens:", test_ids)
print("Decoded prompt:", sp.decode(test_ids))

answer = generate_answer_from_ids(test_ids, max_new_tokens=100, temperature=0.5)
print("Generated answer:", answer)

Prompt text: नमस्ते
Prompt tokens: [20776]
Decoded prompt: नमस्ते
Generated answer: ! आप एक अच्छा था। ⁇ एक अच्छा था। एक अच्छा है। ⁇ 2 था। ⁇  1 का एक अच्छा था। ⁇ 0 था। एक अच्छा था। ⁇  3 था। ⁇ 9 हूँ। एक अच्छा था। एक अच्छा था। एक अच्छा था। एक अच्छा था। एक अच्छा एक अच्छा एक अच्छा था। एक अच्छा एक अच्छा था। एक अच्छा था। एक अच्छा एक अच्छा है। एक अच्छा एक अच्छा एक अच्छा एक अच्छा था। एक अच्छा एक अच्छा एक अच्छा था।


In [119]:
# print("Prompt text:", prompt_text)
# print("Prompt tokens:", prompt_ids)
# print("Decoded prompt:", sp.decode(prompt_ids))

answer = generate_answer_from_ids(prompt_ids, max_new_tokens=100, temperature=0.5)
print("Generated answer:", answer)

Generated answer: ⁇ 8 कि "मुझे एक ऐसी जानकारी चाहिए। "यह एक उचित और क्या आप किस प्रकार के बीच हैं? ⁇ 97 "एक उपयुक्त, या तो यह है। 9. 1,19 है। 9. 2,24 है। 9. 1 है। 9. 1 है। 9. 1,19 है। 9. 8 है। 9. 1,19 है। 9. आप ए है। ⁇ 9? 9 नहीं है। 9? 9 नहीं है


In [114]:
prompt_text = "महात्मा गांधी का जन्म कब हुआ था?"
prompt_ids = sp.encode(prompt_text)
prompt_ids

[5209, 1187, 40, 1424, 2438, 382, 132, 28910]

In [115]:
EOS_ID = sp.eos_id()

@torch.no_grad()
def generate(
    model,
    input_ids,
    max_new_tokens=300,
    temperature=0.6,
    top_p=0.75,
    repetition_penalty=1.5,
    top_k=50
):
    model.eval()

    for _ in range(max_new_tokens):
        logits = model(input_ids)
        next_logits = logits[:, -1, :]

        # Repetition penalty
        if repetition_penalty != 1.0:
            for i in range(input_ids.size(0)):
                unique_tokens = torch.unique(input_ids[i])
                for token_id in unique_tokens:
                    if next_logits[i, token_id] < 0:
                        next_logits[i, token_id] *= repetition_penalty
                    else:
                        next_logits[i, token_id] /= repetition_penalty

        # Temperature scaling
        next_logits = next_logits / temperature

        probs = torch.softmax(next_logits, dim=-1)

        # Top-k filtering
        if top_k > 0:
            top_k_vals, top_k_indices = torch.topk(probs, top_k)
            probs_zeroed = torch.zeros_like(probs).scatter_(-1, top_k_indices, top_k_vals)
            probs = probs_zeroed / top_k_vals.sum(dim=-1, keepdim=True)

        # Top-p nucleus filtering
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = False
        sorted_probs[sorted_indices_to_remove] = 0.0

        # Renormalize
        if sorted_probs.sum() == 0:
            next_token = torch.argmax(probs, dim=-1, keepdim=True)
        else:
            sorted_probs /= sorted_probs.sum(dim=-1, keepdim=True)
            next_token = torch.multinomial(sorted_probs, num_samples=1)
            next_token = sorted_indices.gather(-1, next_token)

        input_ids = torch.cat([input_ids, next_token], dim=1)

        # अगर EOS token हो तो ब्रेक (अगर defined है)
        if next_token.item() == EOS_ID:
            break

    return input_ids

In [116]:
answer = generate_answer_from_ids(prompt_ids, max_new_tokens=50, temperature=0.5)
print("Generated answer:", answer)

Generated answer: ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ । ⁇ ।


In [117]:
prompt_text = "फ्रांस की राजधानी क्या है?"
prompt_ids = sp.encode(prompt_text)

answer = generate_answer_from_ids(
    prompt_ids,
    max_new_tokens=50,
    temperature=0.5
)

print("Generated answer:", answer)

Generated answer: ⁇ 7777777777777777777777777777777777777777777777777
